<a href="https://colab.research.google.com/github/gecrooks/quantumflow-dev/blob/gec085-dev/notebooks/transpile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QuantumFlow Tutorial: Transpiling

[QuantumFlow](https://https://github.com/gecrooks/quantumflow-dev) can convert circuits between a number of popular quantum libraries, such as `qiskit`, `braket`, and `cirq`. These additional libaries can be installed with the `[extra]` or `[dev]` options, e.g. `pip install quantumflow[ext]`

In [ ]:
!pip install --quiet git+https://github.com/gecrooks/quantumflow.git#egg=quantumflow[ext]

Lets built a simple quantum circuit to make a 5 qubit Bell state using IBM's `qiskit`.

In [35]:
import qiskit
qr = qiskit.QuantumRegister(5)
qiskit_circuit = qiskit.QuantumCircuit(qr)
qiskit_circuit.h(qr[0])
qiskit_circuit.cx(qr[0], qr[1])
qiskit_circuit.cx(qr[1], qr[2])
qiskit_circuit.cx(qr[2], qr[3])
qiskit_circuit.cx(qr[3], qr[4])
print(qiskit_circuit)


      ┌───┐                    
q5_0: ┤ H ├──■─────────────────
      └───┘┌─┴─┐               
q5_1: ─────┤ X ├──■────────────
           └───┘┌─┴─┐          
q5_2: ──────────┤ X ├──■───────
                └───┘┌─┴─┐     
q5_3: ───────────────┤ X ├──■──
                     └───┘┌─┴─┐
q5_4: ────────────────────┤ X ├
                          └───┘


We can then convert this `qiskit` circuit to native `quantumflow`,

In [36]:
import quantumflow as qf
qf_circuit = qf.transpile(qiskit_circuit)
qf_circuit

or to Google's `cirq`,

In [37]:
cirq_circuit = qf.transpile(qiskit_circuit, output_format="cirq")
cirq_circuit

0: ───H───@───────────────
          │
1: ───────X───@───────────
              │
2: ───────────X───@───────
                  │
3: ───────────────X───@───
                      │
4: ───────────────────X───

or Amazon's `braket`.

In [38]:
braket_circuit = qf.transpile(qiskit_circuit, output_format="braket")
print(braket_circuit)

T  : |0|1|2|3|4|
                
q0 : -H-C-------
        |       
q1 : ---X-C-----
          |     
q2 : -----X-C---
            |   
q3 : -------X-C-
              | 
q4 : ---------X-

T  : |0|1|2|3|4|


We can also transpile to `quirk`. Click on the link below to load the circuit in the web editor.

In [39]:
quirk_circuit = qf.transpile(qiskit_circuit, output_format="quirk")
print(quirk_circuit)

{"cols":[["H"],["•","X"],[1,"•","X"],[1,1,"•","X"],[1,1,1,"•","X"]]}


In [42]:
from quantumflow import xquirk
print(xquirk.quirk_url(quirk_circuit, escape="True"))

https://algassert.com/quirk#circuit=%7B%22cols%22%3A%5B%5B%22H%22%5D%2C%5B%22%E2%80%A2%22%2C%22X%22%5D%2C%5B1%2C%22%E2%80%A2%22%2C%22X%22%5D%2C%5B1%2C1%2C%22%E2%80%A2%22%2C%22X%22%5D%2C%5B1%2C1%2C1%2C%22%E2%80%A2%22%2C%22X%22%5D%5D%7D


There are a couple of complications to transpiling circuits between libraries. The first problem is the labelling of qubits. QuantumFlow allows a wide variety of labels, including integers, strings, and tuples of integers or strings. In general any hashable, sortable python type. (But you shouldn't mix qubit label types in the same circuit.) However, other libraries are less flexible. Your best bet is to stick to non-negative integers.

The second complication is that different quantum librarise implement different sets of quantum gates. For instance, lets add a controlled-Hadamard gate to our initial qiskit circuit,

In [43]:
qiskit_circuit.ch(qr[3], qr[4])
print(qiskit_circuit)

      ┌───┐                         
q5_0: ┤ H ├──■──────────────────────
      └───┘┌─┴─┐                    
q5_1: ─────┤ X ├──■─────────────────
           └───┘┌─┴─┐               
q5_2: ──────────┤ X ├──■────────────
                └───┘┌─┴─┐          
q5_3: ───────────────┤ X ├──■────■──
                     └───┘┌─┴─┐┌─┴─┐
q5_4: ────────────────────┤ X ├┤ H ├
                          └───┘└───┘


and then transpile to `cirq`.

In [44]:
cirq_circuit = qf.transpile(qiskit_circuit, output_format="cirq")
cirq_circuit

0: ───H───@─────────────────────────────────────────────────
          │
1: ───────X───@─────────────────────────────────────────────
              │
2: ───────────X───@─────────────────────────────────────────
                  │
3: ───────────────X───@───────────────@─────────────────────
                      │               │
4: ───────────────────X───S───H───T───X───T^-1───H───S^-1───

Google's `cirq` does not implement the controlled-Hadamard gate (CH), so QuantumFlow translates the CH gate into an equivalant sequence of gates that `cirq` does understand.

QuantumFlow contains a rich set of gates, so that transpiling to QF is generally a direct literal gate-for-gate translation.

The collection of currently supported transpiled formats can be found in `qf.TRANSPILE_FORMATS`.

In [45]:
qf.TRANSPILE_FORMATS

('qasm',
 'cirq',
 'braket',
 'pyquil',
 'qiskit',
 'quirk',
 'qsim',
 'quantumflow',
 'qutip')